In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time

import re

In [2]:
from bs4 import BeautifulSoup

In [3]:
# 웹드라이버 접속
driver = webdriver.Chrome(executable_path=r'C:\Users\jennie\Desktop\BOAZ\chromedriver_win32\chromedriver.exe')

In [10]:
# 접속할 url 불러오기
df = pd.read_csv('노원_url수집_5개.csv'); df.head()

,Unnamed: 0,Unnamed: 0.1,상호명,상권업종대분류명,표준산업분류명,시군구명,법정동명,도로명주소,위도,경도,검색어,역 명,거리,naver_keyword,naver_map_url
0,0,8,미가족발,음식,한식 음식점업,노원구,공릉동,서울특별시 노원구 동일로193길 19,37.625793,127.071582,공릉동 미가족발,7호선 공릉(서울과학기술대),110.998565,공릉동%20미가족발,https://pcmap.place.naver.com/restaurant/38323...
1,1,10,파란파랑,음식,한식 음식점업,노원구,상계동,서울특별시 노원구 동일로217길 58,37.653247,127.057901,상계동 파란파랑,7호선 노원,254.794028,상계동%20파란파랑,https://pcmap.place.naver.com/restaurant/36215...
2,2,11,전주백반,음식,한식 음식점업,노원구,공릉동,서울특별시 노원구 동일로173가길 101,37.622752,127.072118,공릉동 전주백반,7호선 공릉(서울과학기술대),351.645144,공릉동%20전주백반,https://pcmap.place.naver.com/restaurant/20919...
3,3,26,와우곱창,음식,한식 음식점업,노원구,공릉동,서울특별시 노원구 공릉로 107,37.619985,127.077749,공릉동 와우곱창,7호선 태릉입구,251.418936,공릉동%20와우곱창,https://pcmap.place.naver.com/restaurant/20877...
4,4,28,어촌,음식,일식 음식점업,노원구,월계동,서울특별시 노원구 석계로8가길 29,37.615406,127.064904,월계동 어촌,1호선 석계,65.926456,월계동%20어촌,https://pcmap.place.naver.com/restaurant/10483...


In [59]:
# 수집할 정보들
rating_list = []
user_review_id = {}
review_json = {}
image_json = {}

In [65]:
# rating_df : UserID, ItemID, Rating, Timestamp
# user_review_id : {'공릉역 공릉순두부':{'userid':'reviewid'}, '공릉역 찬이네':{'식당id':'userid','userid',,,},,,}
# review_json : {'공릉역 공릉순두부':{'userid':'review내용'},{'userid':'review내용'},{'userid':'review내용'},,,}
# image_json



for i in range(len(df)):
    
    print('======================================================')
    print('노원구 '+str(i)+'번째 식당')
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naver_map_url'][i])
    thisurl = df['naver_map_url'][i]
    
    # 더보기 버튼 다 누르기
    # 더보기 버튼 누르기까지 10개
    # 더보기 버튼 누르면 10개 추가됨
    while True:
        try:
            driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[5]/div[4]/div[4]/div[2]/a').click()
            time.sleep(2)
        except:
            print('-더보기 버튼 모두 클릭 완료-')
            break
    
    # 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(1)
    
    # 식당 구분
    restaurant_name = df['법정동명'][i]+' '+df['상호명'][i]
    print('식당 이름 : '+restaurant_name)
    
    user_review_id[restaurant_name] = {}
    review_json[restaurant_name] = {}
    image_json[restaurant_name] = {}
    
    try:
        restaurant_classificaton = soup.find_all('span',attrs = {'class':'_3ocDE'})[0].text
    except:
        restaurant_classificaton = 'none'
    print('식당 구분 : '+restaurant_classificaton)
    print('----------------------------------------------')
    
    try:
        one_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수
        print('리뷰 총 개수 : '+str(review_num))
    
        # 리뷰 개수
        for i in range(len(one_review)):
        
            # user url
            user_url = one_review[i].find('div', attrs = {'class':'_23Rml'}).find('a').get('href')
            print('user_url = '+user_url)
            
            # user url로부터 user code 뽑아내기
            user_code = re.findall(r"my/(\w+)", user_url)[0]
            print('user_code = '+user_code)
            
            # review 1개에 대한 id 만들기 
            res_code = re.findall(r"restaurant/(\d+)", thisurl)[0]
            review_id = str(res_code)+"_"+user_code
            print('review_id = '+review_id)
            
            # rating, 별점
            rating = one_review[i].find('span', attrs = {'class':'_2tObC'}).text
            print('rating = '+rating)
            
            # 주의!!! 사진 리뷰 유무에 따라 날짜 파싱코드 다름
            # ('span', attrs = {'class':'_3WqoL'})
            # 사진 없는 경우 : 총 6개 중 4번째
            # 사진 있는 경우 : 총 5개 중 3번째
            
            # date
            # 사진 리뷰 없음
            if len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 5:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[2].text
            elif len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 6:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[3].text
            else:
                date = "날짜 정보 모르것는디"
            print('date = '+date)
            
            # review 내용
            try : 
                review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text
            except: # 리뷰가 없다면
                review_content = "리뷰 내용 없음"
            print('리뷰 내용 : '+review_content)
            print('----------------------------------------------')
            print('\n')
            
            
            
            
            #리뷰정보 = restaurant_name, restaurant_classificaton, user_url, user_code, review_num, rating, nth_visit, date, review_content

            # user_review_id
            user_review_id[restaurant_name][user_code] = review_id
            
            # review_json
            review_json[restaurant_name][review_id] = review_content
            
            # rating_df_list
            naver_review = user_code, restaurant_name, rating, date
            rating_list.append(naver_review)
    
    
    except NoSuchElementException:
            none_review = "네이버 리뷰 없음"
            print(none_review)
            review_num = 0
            #리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review
            #review_list.append(리뷰정보)
            
            # rating_df_list
            naver_review = user_code, restaurant_name, none_review, none_reivew
            rating_list.append(naver_review)
            
            
    print('\n')

노원구 0번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 공릉동 미가족발
식당 구분 : 족발,보쌈
----------------------------------------------
리뷰 총 개수 : 0


노원구 1번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 상계동 파란파랑
식당 구분 : 카페,디저트
----------------------------------------------
리뷰 총 개수 : 10
user_url = https://m.place.naver.com/my/5fb281bfb61cc4c3dd9bcc20/review
user_code = 5fb281bfb61cc4c3dd9bcc20
review_id = 36215455_5fb281bfb61cc4c3dd9bcc20
rating = 4.5
date = 2021.03.09
리뷰 내용 : 블루베리 음료 맛나요
----------------------------------------------


user_url = https://m.place.naver.com/my/5e1c524b8f87a842bc580991/review
user_code = 5e1c524b8f87a842bc580991
review_id = 36215455_5e1c524b8f87a842bc580991
rating = 5
date = 2021.01.27
리뷰 내용 : 좋아요
----------------------------------------------


user_url = https://m.place.naver.com/my/5e2afda88f87a842bc86c828/review
user_code = 5e2afda88f87a842bc86c828
review_id = 36215455_5e2afda88f87a842bc86c828
rating = 5
date = 2021.01.24
리뷰 내용 : 커피맛나요~
----------------------------------------------


user_url =

In [48]:
rating_df = pd.DataFrame(rating_list, columns = ['UserID', 'ItemID', 'Rating', 'Timestamp'])
rating_df.head()

,UserID,ItemID,Rating,Timestamp
0,5fb281bfb61cc4c3dd9bcc20,상계동 파란파랑,4.5,2021.03.09
1,5e1c524b8f87a842bc580991,상계동 파란파랑,5,2021.01.27
2,5e2afda88f87a842bc86c828,상계동 파란파랑,5,2021.01.24
3,5e2afda88f87a842bc86c828,상계동 파란파랑,5,2020.11.02
4,5c74c9695e722ae418d30dd9,상계동 파란파랑,5,2020.10.06


In [64]:
user_review_id

{'공릉동 미가족발': {},
 '상계동 파란파랑': {'5fb281bfb61cc4c3dd9bcc20': '36215455_5fb281bfb61cc4c3dd9bcc20',
  '5e1c524b8f87a842bc580991': '36215455_5e1c524b8f87a842bc580991',
  '5e2afda88f87a842bc86c828': '36215455_5e2afda88f87a842bc86c828',
  '5c74c9695e722ae418d30dd9': '36215455_5c74c9695e722ae418d30dd9',
  '5ee80a238f87a842bc994c6b': '36215455_5ee80a238f87a842bc994c6b',
  '5e3ccabb8f87a842bc8ffb72': '36215455_5e3ccabb8f87a842bc8ffb72'},
 '공릉동 전주백반': {'5e08b8078f87a842bc3d8462': '20919106_5e08b8078f87a842bc3d8462',
  '5bf26bba148347e086d7536b': '20919106_5bf26bba148347e086d7536b',
  '5e26231e8f87a842bc7f5d75': '20919106_5e26231e8f87a842bc7f5d75',
  '5e92b2278f87a842bc0cacba': '20919106_5e92b2278f87a842bc0cacba',
  '5c210660b51390e1c13c1453': '20919106_5c210660b51390e1c13c1453',
  '5c1a58cbbbecf48470242b2f': '20919106_5c1a58cbbbecf48470242b2f',
  '5d8807428f87a842bc08365c': '20919106_5d8807428f87a842bc08365c',
  '5cd13e5d8f87a842bc326a01': '20919106_5cd13e5d8f87a842bc326a01'},
 '공릉동 와우곱창': {'5ee1

In [66]:
review_json

{'공릉동 미가족발': {},
 '상계동 파란파랑': {'36215455_5fb281bfb61cc4c3dd9bcc20': '블루베리 음료 맛나요',
  '36215455_5e1c524b8f87a842bc580991': '좋아요',
  '36215455_5e2afda88f87a842bc86c828': '집앞이라 일주일에 두세번은 가는거 같아요 맛있어서 가는거에요!',
  '36215455_5c74c9695e722ae418d30dd9': '맛있어요',
  '36215455_5ee80a238f87a842bc994c6b': '굿',
  '36215455_5e3ccabb8f87a842bc8ffb72': '굿'},
 '공릉동 전주백반': {'20919106_5e08b8078f87a842bc3d8462': '음식도 맛있구~\n사장님 엄청 친절 하세요~',
  '20919106_5bf26bba148347e086d7536b': '맛있음',
  '20919106_5e26231e8f87a842bc7f5d75': '점심 고민하지 마세요^^',
  '20919106_5e92b2278f87a842bc0cacba': '괜찮아요',
  '20919106_5c210660b51390e1c13c1453': '맛있어요',
  '20919106_5c1a58cbbbecf48470242b2f': '맛나요',
  '20919106_5d8807428f87a842bc08365c': '저렴하거 양도 많고 무엇보다 맛나요^^',
  '20919106_5cd13e5d8f87a842bc326a01': '오로지 백반만 하는곳\n아침5시~저녁7시까지 ㅡ월~금\n'},
 '공릉동 와우곱창': {'20877247_5ee1a5488f87a842bc2ca77d': 'ㆍ',
  '20877247_5c3701444fd00c134bb94913': '매워요',
  '20877247_5be8ff37d4de015359ab1fc7': '좋아요',
  '20877247_5e6e4ec58f87a842bcba3e98': '찐맛!!지나가다 들

In [ ]:
## 중간에 멈췄을 때, 다시 돌리기 용
## range에 시작할 index부터 넣어주면 됨

driver = webdriver.Chrome(executable_path=r'C:\Users\jennie\Desktop\BOAZ\chromedriver_win32\chromedriver.exe')

for i in range(len(df)):
    
    print('======================================================')
    print('노원구 '+str(i)+'번째 식당')
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naver_map_url'][i])
    
    
    # 더보기 버튼 다 누르기
    # 더보기 버튼 누르기까지 10개
    # 더보기 버튼 누르면 10개 추가됨
    while True:
        try:
            driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[5]/div[4]/div[4]/div[2]/a').click()
            time.sleep(2)
        except:
            print('-더보기 버튼 모두 클릭 완료-')
            break
    
    # 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(1)
    
    # 식당 구분
    restaurant_name = df['법정동명'][i]+' '+df['상호명'][i]
    print('식당 이름 : '+restaurant_name)
    try:
        restaurant_classificaton = soup.find_all('span',attrs = {'class':'_3ocDE'})[0].text
    except:
        restaurant_classificaton = 'none'
    print('식당 구분 : '+restaurant_classificaton)
    print('----------------------------------------------')
    
    try:
        one_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수
        print('리뷰 총 개수 : '+str(review_num))
    
        # 리뷰 개수
        for i in range(len(one_review)):
        
            # user url
            user_url = one_review[i].find('div', attrs = {'class':'_23Rml'}).find('a').get('href')
            print('user_url = '+user_url)
            
            # user url로부터 user code 뽑아내기
            user_code = re.findall(r"my/(\w+)", user_url)[0]
            print('user_code = '+user_code)
            
            # rating, 별점
            rating = one_review[i].find('span', attrs = {'class':'_2tObC'}).text
            print('rating = '+rating)
            
            # 주의!!! 사진 리뷰 유무에 따라 날짜 파싱코드 다름
            # ('span', attrs = {'class':'_3WqoL'})
            # 사진 없는 경우 : 총 6개 중 4번째
            # 사진 있는 경우 : 총 5개 중 3번째
            
            # date
            # 사진 리뷰 없음
            if len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 5:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[2].text
            elif len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 6:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[3].text
            else:
                date = "날짜 정보 모르것는디"
            print('date = '+date)
            
            # review 내용
            try : 
                review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text
            except: # 리뷰가 없다면
                review_content = "리뷰 내용 없음"
            print('리뷰 내용 : '+review_content)
            print('----------------------------------------------')
            print('\n')
    
            
            
            #리뷰정보 = restaurant_name, restaurant_classificaton, user_url, user_code, review_num, rating, nth_visit, date, review_content
            #review_list.append(리뷰정보)
            
            # rating_df_list
            naver_review = user_code, restaurant_name, rating, date
            rating_list.append(naver_review)
    
    
    except NoSuchElementException:
            none_review = "네이버 리뷰 없음"
            print(none_review)
            review_num = 0
            #리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review
            #review_list.append(리뷰정보)
            
            # rating_df_list
            naver_review = user_code, restaurant_name, none_review, none_reivew
            rating_list.append(naver_review)
            
            
    print('\n')